In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir("/root/FYP")
import json
import ast
import numpy as np
import pandas as pd
import itertools
from pprint import pprint
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from src.analysis import analyse_code
import seaborn as sns

In [ ]:
# import matplotlib
# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

In [ ]:
DATASETS_ROOT = "datasets"

def generate_tasks_df():
    with open(os.path.join(DATASETS_ROOT, "dataset.formatted.json")) as f:
        notebooks = json.loads(f.read())
    with open(os.path.join("resources", "task_return_types.json")) as f:
        return_types = json.loads(f.read())
    tasks = []
    for notebook in notebooks:
        for i, turn in enumerate(notebook["turns"]):
            task = {
                "dataset_src": notebook["dataset_src"],
                "notebook_name": notebook["notebook_name"],
                "turn_index": i,
                "code_context": turn["input"],
                "intent": turn["turn"]["intent"]["value"],
                "code": turn["turn"]["code"]["value"],
                "return_types": return_types[str((notebook["dataset_src"], notebook["notebook_name"], i))]
            }
            task |= analyse_code(task["code_context"], task["code"])
            task["task_type"] = task["task_type"].split(".")[-1]
            task["dataset_src"] = dict({"existing_tasks": "Existing Tasks", "new_tasks": "New Tasks"})[task["dataset_src"]]
            tasks.append(task)
    tasks_path = os.path.join("resources", "metadata.json")
    df = pd.DataFrame(tasks)
    df.to_json(tasks_path)
    return df

df = generate_tasks_df()

In [ ]:
def barplot(data, y, x, hue=None):
    fig, ax = plt.subplots(figsize=(4, 3))
    fig.set_size_inches(w=5.90666, h=4.5)
    sns.barplot(data=data, x=x, y=y, hue=hue, palette="tab10", ax=ax)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(f'{x} - {y}')
    plt.legend(title=hue)
    if data[x].nunique() > 3:
        plt.xticks(rotation=90)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.show()

def facetgridplot(data, x, y, z, hue):
    plt.figure(figsize=(12, 8))
    g = sns.FacetGrid(data, col=z, col_wrap=2)
    g.map(sns.barplot, x, y, hue, palette="tab10")
    if data[x].nunique() > 3:
        g.set_xticklabels(rotation=90)
    g.add_legend(title=hue)
    g.set_axis_labels(x, y)
    g.set_titles(col_template="{col_name}")
    plt.show()

def generate_bin_col(data, bin_col, bin_size):
    bin_edges = np.arange(0, data[bin_col].max() + bin_size, bin_size)
    data[f"{bin_col}_bin"] = pd.cut(data[bin_col], bins=bin_edges, right=False, labels=[f"{i}-{i+bin_size-1}" for i in bin_edges[:-1]])
    return data

def histplot(data, xlabel, ylabel="Frequency Density", cntlabel="size", bin_edges=None):
    plt.figure(figsize=(4, 3))
    data = data.copy()
    colors = {
        'Existing Tasks': 'blue',
        'New Tasks': 'green',
    }

    if bin_edges is None:
        bin_edges = np.arange(min(data[cntlabel])-0.5, max(data[cntlabel])+1.5, 1)

    for dataset_src in data['dataset_src'].unique():
        subset = data[data['dataset_src'] == dataset_src]
        
        # Calculate density
        density, _ = np.histogram(subset[cntlabel], bins=bin_edges, density=True)
        
        # Plot density histogram
        plt.bar(bin_edges[:-1], density, width=np.diff(bin_edges), 
                alpha=0.5, edgecolor="k", label=dataset_src, 
                color=colors.get(dataset_src), align='edge')

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(title='Dataset Source')
    plt.tight_layout()
    plt.show()
    

In [ ]:
print("\nTotal Tasks: ", len(df))
print("\nTotal Notebooks: ", len(df.groupby("notebook_name")))
print("\nTasks per", df.groupby("dataset_src").size())
print("\nNotebooks per", df.groupby(["dataset_src","notebook_name"]).size().groupby(level=0).size())


dataset_summary = pd.DataFrame({
    'Tasks': df.groupby("dataset_src").size(),
    'Notebooks': df.groupby(["dataset_src", "notebook_name"]).size().groupby(level=0).size()
})

# Reset the index to make 'dataset_src' a column
dataset_summary = dataset_summary.reset_index()

dataset_summary['Tasks per Notebook'] = dataset_summary['Tasks'] / dataset_summary['Notebooks']

# Rename the 'index' column to 'Dataset'
dataset_summary = dataset_summary.rename(columns={'dataset_src': 'Dataset'})
dataset_summary['Tasks per Notebook'] = dataset_summary['Tasks per Notebook'].round(2)

# Display the dataset summary
print(dataset_summary.to_latex(index=False))

In [ ]:
task_count = df.groupby(["dataset_src", "notebook_name"]).size().reset_index(name='size')

histplot(
    data=task_count,
    xlabel='Num. Notebook Problems',
)


In [ ]:
plt.savefig('figures/num_tasks.pgf')

In [ ]:
call_count = lambda calls: sum([len(v) for v in calls.values()])
df["size"] = df["modules"].apply(call_count)
histplot(
    data=df,
    xlabel="Num. Pandas API Calls",
)


In [ ]:
plt.savefig('figures/num_calls.pgf')

In [ ]:
agg_imports = lambda s: {i.split(".")[0] for i in set.union(*s)}
df_notebook_imports = df.groupby("notebook_name")["imports"].agg(agg_imports)
import_count = Counter(itertools.chain(*map(list, df_notebook_imports.values)))
to_show = 10
sorted_values = sorted(import_count.values(), reverse=True)[:to_show]
sorted_keys = sorted(import_count, key=import_count.get, reverse=True)[:to_show]
df_res = pd.DataFrame({"imports": sorted_keys, "freq": sorted_values})

barplot(
    data=df_res,
    x="imports",
    y="freq"
)

In [ ]:
from src.analysis import PANDAS_METHODS

pandas_calls = Counter(itertools.chain(*map(lambda m: m["pandas"], df["modules"].values)))
pandas_calls = {k: v for k, v in pandas_calls.items() if k in PANDAS_METHODS}

to_display = 20
sorted_values = sorted(pandas_calls.values(), reverse=True)[:to_display]
sorted_keys = sorted(pandas_calls, key=pandas_calls.get, reverse=True)[:to_display]
df_res = pd.DataFrame({"calls": sorted_keys, "freq": sorted_values})

barplot(
    data=df_res,
    x="calls",
    y="freq"
)

In [ ]:
task_return_types = Counter(itertools.chain(*df["return_types"].values))
sorted_values = sorted(task_return_types.values(), reverse=True)
sorted_keys = sorted(task_return_types, key=task_return_types.get, reverse=True)
df_res = pd.DataFrame({"return_types": sorted_keys, "freq": sorted_values})
df_res["percentage"] = (df_res["freq"] / df_res["freq"].sum()) * 100

barplot(
    data=df_res,
    x="return_types",
    y="freq"
)

In [ ]:
from src.prompt_utils import get_num_tokens

df["num_tokens"] = df["code_context"].apply(get_num_tokens)

histplot(
    data=df,
    xlabel="Num. Tokens",
    cntlabel="num_tokens",
    bin_edges = np.arange(0, 4000, 200)
)

In [ ]:
def calc_ratio_md_code(r):
    cc = r["code_context"]
    md = "\n".join([ss for s in cc.split("# In[ ]:") for ss in s.split("\n") if ss.startswith("#")])
    code = "\n".join([ss for s in cc.split("# In[ ]:") for ss in s.split("\n") if ss.strip() and not ss.startswith("#")])
    e = 0.000000000000000000000000000000001
    return len(md)
    # return (r["size"] +e)/(len(md) +e)

df["md_code_ratio"] = df.apply(calc_ratio_md_code, axis=1)

histplot(
    data=df,
    xlabel="Markdown",
    cntlabel="md_code_ratio",
    bin_edges = np.arange(0, 3000, 50)
)


In [ ]:
from src.prompt_utils import get_num_tokens

df["num_lines"] = df["code_context"].apply(lambda s: len([ss for ss in s.split("\n") if ss.strip() and not ss.strip().startswith("#")]))

histplot(
    data=df,
    xlabel="Num. Tokens",
    cntlabel="num_lines",
    bin_edges = np.arange(0, 200, 5)
)

In [ ]:
df["dataframes_tot"] = df["dataframes"].apply(lambda l: len(set(l)))

histplot(
    data=df,
    xlabel="Num. Dataframes/Series",
    cntlabel="dataframes_tot",
    bin_edges = np.arange(0, 30, 1)
)

In [ ]:
plt.savefig('figures/num_dataframes.pgf')

In [ ]:
# plt.savefig('figures/num_tokens.png')

In [ ]:
print(df_res[["return_types", "percentage"]].round(2).to_latex(index=False))

In [ ]:
with open("/root/FYP/experiments/arcade.CoT-FS+EXP/predictions.llama3_instruct_70b.json") as f:
    dataset = json.loads(f.read())

from src.multistep import parse_outputs

data = {}

return_type_map = {
    'int': "Numeric",
    'float': "Numeric",
    'str': "String",
    'NoneType': "Empty",
    'list': "List/Tuple/Set",
    'bool': "Boolean",
    'tuple': "List/Tuple/Set",
    'set': "List/Tuple/Set"
}

def get_output_type(v):
    parsed_output = parse_outputs(v)
    if isinstance(parsed_output, str) :
        if "dtype" in parsed_output:
            return "Series"
        elif "Figure" in parsed_output:
            return "Plot"
    t = type(parsed_output).__name__
    return return_type_map.get(t, t)

for n in dataset:
    for i, t in enumerate(n["turns"]):
        data[str((n["metadata"]["dataset_src"], n["metadata"]["notebook_name"], i))] = [get_output_type(v) for v in t.get("ref_output_html", {}).values()]

with open("resources/task_return_types.json", "w") as f:
    f.write(json.dumps(data, indent=2))

In [ ]:
method_groups = {
    "aggregation": {
        "methods": {"groupby", "agg", "Grouper"}
    },
    "transformation": {
        "apply": {"apply", "applymap", "map"},
        "reshape": {"pivot_table", "pivot", "melt", "stack", "unstack", "transpose"},
        "bin": {"cut", "qcut"},
        "explode": {"explode"},
        "time_series": {"shift", "pct_change"},
        "compute": {"cumsum", "diff", "rank"}
    },
    "combination": {
        "methods": {"join", "merge", "concat", "append"}
    },
    "selection": {
        "indexing": {"loc", "iloc", "between", "filter", "isin", "isna", "isnull", "notnull", "query", "where"},
        "sorting": {"sort_values", "sort_index"},
        "subset": {"nlargest", "nsmallest", "head", "tail", "first", "last"}
    },
    "cleaning": {
        "missing_data": {"dropna", "fillna", "ffill", "bfill"},
        "duplicates": {"drop_duplicates", "duplicated"},
        "type_conversion": {"astype", "to_numeric", "to_datetime", "to_period", "to_frame", "to_list", "tolist", "ravel"},
        "renaming": {"rename", "rename_axis"},
        "structure": {"reset_index", "set_index", "reindex", "insert"}
    },
    "strings": {
        "methods": {"str", "contains", "extract", "replace", "match", "sub", "startswith", "endswith", "split"}
    },
    "computation": {
        "statistics": {"mean", "max", "min", "median", "mode", "std", "var", "quantile", "describe"},
        "aggregation": {"count", "sum", "nunique", "unique", "value_counts"},
        "correlation": {"corr", "cov"},
        "arithmetic": {"add", "div", "divide", "clip", "round"}
    },
    "datetime": {
        "conversion": {"to_datetime", "to_timedelta"},
        "components": {"dt.year", "dt.month", "dt.day", "dt.hour", "dt.minute", "dt.second"},
        "operations": {"strftime", "strptime", "tz_localize", "tz_convert"},
        "periods": {"to_period", "PeriodIndex", "period_range"},
        "timedelta": {"Timedelta", "timedelta_range"},
        "offsets": {"DateOffset", "BDay", "CDay", "Week", "MonthEnd", "YearEnd"}
    },
    "visualization": {
        "methods": {"plot", "boxplot", "hist", "lmplot", "barplot", "scatter", "lmplot", "barplot", "Figure", "Layout", "Bar", "show", "Scatter"}
    },
}

method_groups_flattened = {category: set(itertools.chain.from_iterable(subcategories.values())) 
                  for category, subcategories in method_groups.items()}

method_group_priority = [
    "aggregation",
    "combination",
    "transformation",
    "computation",
    "selection",
    "cleaning",
    "datetime",
    "strings",
    "visualization"
]

def classify_tasks(row):
    result = {f"is_{category}": False for category in method_groups_flattened.keys()}
    for category, methods in method_groups_flattened.items():
        if any(method in row["code"] for method in methods):
            result[f"is_{category}"] = True
    if any(op in row["code"] for op in set({"*", "/", "+"})):
        result['is_computation'] = True
    if any(op in row["code"] for op in set({"==", ">", "<"})):
        result['is_selection'] = True
    result['task_type'] = "other"
    for category in method_group_priority:
        if result[f"is_{category}"]:
            result['task_type'] = category
            break
    return pd.Series(result)

df["calls"] = df["function_calls"].apply(lambda ms: [ss for s in ms for ss in s.split(".")[1:] if ss])
res = df.apply(classify_tasks, axis=1)
for col in res.columns:
    df[col] = res[col]

In [ ]:
# df = pd.DataFrame(list(method_groups_flattened.items()), columns=['Group', 'Methods'])
# df["Group"] = df["Group"].apply(lambda s: s.capitalize())
# df["Methods"] = df["Methods"].apply(lambda l: f"{', '.join(list(l)[:5])} {', ...' if len(l) > 5 else ''}")
# print(df.to_latex(index=False))

In [ ]:
is_columns = [col for col in df.columns if col.startswith('is_')]
percentage_true = df[is_columns].mean() * 100
result_df = pd.DataFrame({
    'Task Type': percentage_true.index,
    '%': percentage_true.values
}).round(1)
result_df["Task Type"] = result_df["Task Type"].apply(lambda s: s.split("_")[-1].capitalize())
print(result_df.to_latex(index=False))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

is_columns = [col for col in df.columns if col.startswith('is_')]
correlation_matrix = df[is_columns].corr().round(2)
rename_func = lambda x: x.split("_")[-1].capitalize()
correlation_matrix = correlation_matrix.rename(columns=rename_func, index=rename_func)

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
corr_values = correlation_matrix.where(~mask)
vmax = corr_values.max().max()

plt.figure(figsize=(5, 5))
sns.heatmap(
    correlation_matrix, 
    annot=True, 
    cmap='inferno', 
    vmin=-0.3,
    vmax=0.3, 
    center=0.0,
    annot_kws={"size": 8},
    mask=mask
)
plt.tight_layout()
plt.title('Correlation between Pandas Method groups')
plt.savefig('figures/pandas_method_corr.pgf')
plt.show()

In [ ]:
is_columns = [col for col in df.columns if col.startswith('is_')]
percentage_true = df[is_columns].mean() * 100
result_df = pd.DataFrame({
    'Task Type': percentage_true.index,
    '%': percentage_true.values
}).round(1)
result_df["Task Type"] = result_df["Task Type"].apply(lambda s: s.split("_")[-1].capitalize())
print(result_df.to_latex(index=False))

In [ ]:
from src.prompts import select_dataframes_by_priority

code = """
from pandas import Series, DataFrame
import pandas as pd

df = pd.read_csv('NYC_Restaurants.csv', dtype=str)

df_noduplicates = df.drop_duplicates(subset='RESTAURANT')
df_notchains = df_noduplicates.groupby("DBA").filter(lambda x: len(x) == 1)
boro_notchain_pivot = pd.pivot_table(df_notchains, index = 'BORO', values = 'RESTAURANT', aggfunc = lambda x: len(x.unique()))
boro_restaurant_pivot = pd.pivot_table(df_noduplicates, index = 'BORO', values = 'RESTAURANT', aggfunc = lambda x: len(x.unique()))
boro_notchain_pivot['TOTAL RESTAURANTS'] = boro_restaurant_pivot

cuis = df_noduplicates['CUISINE DESCRIPTION'].value_counts()
mask = (df['VIOLATION CODE']).isnull()
no_violations = df[mask]
no_violations[['CUISINE DESCRIPTION', 'RESTAURANT']]

cuisine_no_violations = no_violations['CUISINE DESCRIPTION'].value_counts()
mask = (df['VIOLATION CODE']).notnull()
violations = df[mask]
cuisine_violations = violations['CUISINE DESCRIPTION'].value_counts()

total_cuisine = pd.concat([cuisine_violations , cuisine_no_violations], axis = 1)
violations = pd.crosstab(df['BORO'], df['VIOLATION DESCRIPTION']).query('BORO != ["Missing"]')

vstack = violations.stack()
violations2 = vstack.unstack('BORO')
mostcommon = DataFrame({'Most Common Complaint':violations2.idxmax(),'Number of Complaints':violations2.max()}) 
"""

select_dataframes_by_priority(code, max_dataframes=12, display=True)
plt.savefig('figures/dataframe_dep_graph.pgf')

In [ ]:
print("""
<notebook_context>\n[showing 24 out of 30 notebook cells]\n\n# In[]:\nprint(df_airing)\n\n# Out[]:\n[showing 1 sample rows out of 66 rows]\n                       name (str) studio (str)                theme (str) tags (str) source (str)  rating (float64)  year (float64)                                          synopsis (str) demographic (str) status (str)  eps (float64)  eps_avg_duration_in_min (float64) rated_by (str)  numeric_ratings (float64)\n1955  'Ninja Bear the Animation\u2605'     'KOO-KI'  'Anthropomorphic, School'   'Comedy'   'Original'               NaN          2017.0  'The Ninja International ...icSchool\\nDemographicKids'            'Kids'     'Airing'            NaN                                0.0          '401'                      401.0\n\n# In[]:\nprint(df_anim)\n\n# Out[]:\n[showing 3 sample rows out of 1812 rows]\n                                              theme (str)                 name (str)  rating (float64)\n124   [' Historical', ' Parody', ' Samurai', 'Gag Humor']                 'Gintama''              9.05\n1303                                           ['School']  'Kareshi Kanojo no Jijou'              7.60\n1488                                     ['Mahou Shoujo']  'Delicious Party\u2661Precure'              7.24\n\n# In[]:\nprint(df_dem)\n\n# Out[]:\n[showing 3 sample rows out of 458 rows]\n                             numeric_ratings (float64)\ndemographic studio                                    \nKids        A-1 Pictures                         839.0\nShounen     AXsiZ                             183000.0\n            Kyoto Animation                   779000.0\n\n# In[]:\nprint(df_dur)\n\n# Out[]:\n[showing 3 sample rows out of 2402 rows]\n     source (str)  eps (float64)  eps_avg_duration_in_min (float64)  average_total_duration (float64)\n212       'Manga'          161.0                               24.0                            3864.0\n1877      'Manga'           48.0                                9.0                             432.0\n2342      'Other'           26.0                               24.0                             624.0\n\n# In[]:\nprint(df_tag)\n\n# Out[]:\n[showing 3 sample rows out of 8759 rows]\n        theme (str)   tags (str)\n192   'Super Power'  'Adventure'\n1398       'School'      'Drama'\n1556      ' Shoujo'      ' Life'\n\n# In[]:\n# What is the most common tag associated with each theme?\n\n# In[]:\ndf_tag = df[['theme', 'tags']].dropna()\ndf_tag.theme = df_tag.theme.apply(split_fn)\ndf_tag.tags = df_tag.tags.apply(split_fn)\ndf_tag = df_tag.explode(column=['theme']).explode('tags')\ndf_tag.groupby('theme').agg(pd.Series.mode)\n\n# Out[]:\n[showing 3 sample rows out of 101 rows]\n              tags (ndarray)\ntheme                       \n Historical           Action\nMedical               Comedy\nReverse Harem        Romance\n\n# In[]:\nprint(df_tag)\n\n# Out[]:\n[showing 3 sample rows out of 8759 rows]\n        theme (str)   tags (str)\n192   'Super Power'  'Adventure'\n1398       'School'      'Drama'\n1556      ' Shoujo'      ' Life'\n\n# In[]:\nprint(df_rating)\n\n# Out[]:\n[showing 3 sample rows out of 1898 rows]\n                                     name (str)  numeric_ratings (float64)  rating (float64) theme (str)\n52    'Nanatsu no Taizai: Imashime no Fukkatsu'                   986000.0              7.64   'Shounen'\n1103                                 'Bakuon!!'                    58000.0              6.48     'CGDCT'\n1331                          'Kyou kara Maou!'                   103000.0              7.65    'Isekai'\n\n# In[]:\n# List the average total duration of anime series based on their source.\n\n# In[]:\ndf_dur = df[['source', 'eps', 'eps_avg_duration_in_min']].dropna()\ndf_dur['average_total_duration'] = df.eps * df.eps_avg_duration_in_min\ndf_dur[['source', 'average_total_duration']].groupby('source').mean()\n\n# Out[]:\n[showing 3 sample rows out of 15 rows]\n             average_total_duration (float64)\nsource                                       \nLight novel                            498.30\nMusic                                   90.00\nOther                                  697.98\n\n# In[]:\nprint(df_dur)\n\n# Out[]:\n[showing 3 sample rows out of 2402 rows]\n     source (str)  eps (float64)  eps_avg_duration_in_min (float64)  average_total_duration (float64)\n212       'Manga'          161.0                               24.0                            3864.0\n1877      'Manga'           48.0                                9.0                             432.0\n2342      'Other'           26.0                               24.0                             624.0\n\n# In[]:\nprint(df_studio)\n\n# Out[]:\n[showing 3 sample rows out of 242 rows]\n     studio (str)  rating (float64)\n459   'Bee Train'              7.04\n1500    'Unknown'              5.77\n1728      'dwarf'              6.30\n\n# In[]:\n# What are the most popular studios in terms of number of ratings for each demographic?\n\n# In[]:\ndf_dem = df[['studio', 'demographic', 'numeric_ratings']]\ndf_dem = df_dem.groupby(['demographic', 'studio']).sum().sort_values('numeric_ratings', ascending=False)\ndf_dem.reset_index().groupby('demographic').first()\n\n# Out[]:\n               studio (str)  numeric_ratings (float64)\ndemographic                                           \nJosei            'Madhouse'                  1032000.0\nKids                  'OLM'                  1680859.0\nSeinen           'Madhouse'                 10278900.0\nShoujo       'Brain's Base'                  3296000.0\nShounen             'Bones'                 20615200.0\n\n# In[]:\nprint(df_theme)\n\n# Out[]:\n[showing 3 sample rows out of 3896 rows]\n          theme (str)  rated_by (float64)\n330   ' Martial Arts'            105000.0\n2105           'Kids'               267.0\n2414           'Kids'                73.0\n\n# In[]:\nprint(df_dem)\n\n# Out[]:\n[showing 3 sample rows out of 458 rows]\n                             numeric_ratings (float64)\ndemographic studio                                    \nKids        A-1 Pictures                         839.0\nShounen     AXsiZ                             183000.0\n            Kyoto Animation                   779000.0\n\n# In[]:\n# List the highest rated anime for each theme.\n\n# In[]:\ndf_anim = df[['theme', 'name', 'rating']].dropna()\ndf_anim.theme = df_anim.theme.apply(split_fn)\ndf_anim.sort_values(by='rating', ascending=False).explode('theme').groupby('theme').first()\n\n# Out[]:\n[showing 3 sample rows out of 100 rows]\n                                                  name (str)  rating (float64)\ntheme                                                                         \n High Stakes Game  'Gyakkyou Burai Kaiji: Ultimate Survivor'              8.28\nMedical                                    'Kuuchuu Buranko'              7.94\nReverse Harem                'Yamato Nadeshiko Shichihenge\u2665'              7.73\n\n# In[]:\nprint(df_anim)\n\n# Out[]:\n[showing 3 sample rows out of 1812 rows]\n                                              theme (str)                 name (str)  rating (float64)\n124   [' Historical', ' Parody', ' Samurai', 'Gag Humor']                 'Gintama''              9.05\n1303                                           ['School']  'Kareshi Kanojo no Jijou'              7.60\n1488                                     ['Mahou Shoujo']  'Delicious Party\u2661Precure'              7.24\n\n# In[]:\n# What are the top 3 highest rated anime still on air?\n\n# In[]:\nprint(df)\n\n# Out[]:\n[showing 1 sample rows out of 3005 rows]\n               name (str) studio (str)  theme (str)                                 tags (str) source (str)  rating (float64)  year (float64)                                          synopsis (str) demographic (str) status (str)  eps (float64)  eps_avg_duration_in_min (float64) rated_by (str)\n226  'Ushio to Tora (TV)'      'MAPPA'  'Mythology'  'Action, Adventure, Comedy, Supernatural'      'Manga'              7.58          2015.0  'Ushio Aotsuki is a stubb...ology\\nDemographicShounen'         'Shounen'   'Finished'           26.0                               24.0         '207K'\n\n# In[]:\n\ndf_airing = df[df.status == 'Airing']\ndf_airing[['name', 'rating']].sort_values(by='rating', ascending=False).head(3)\n\n# Out[]:\n                                        name (str)  rating (float64)\n119                                 'Spy x Family'              9.09\n927   'Kaguya-sama wa Kokurasetai: Ultra Romantic'              8.97\n1167                          'Kingdom 4th Season'              8.69\n\n# In[]:\nprint(df_airing)\n\n# Out[]:\n[showing 1 sample rows out of 66 rows]\n                       name (str) studio (str)                theme (str) tags (str) source (str)  rating (float64)  year (float64)                                          synopsis (str) demographic (str) status (str)  eps (float64)  eps_avg_duration_in_min (float64) rated_by (str)  numeric_ratings (float64)\n1955  'Ninja Bear the Animation\u2605'     'KOO-KI'  'Anthropomorphic, School'   'Comedy'   'Original'               NaN          2017.0  'The Ninja International ...icSchool\\nDemographicKids'            'Kids'     'Airing'            NaN                                0.0          '401'                      401.0\n</notebook_context>
""")

In [ ]:
print("# Columns in df_anim with example values:\n# Id (addam-marbrand), Label (Addam Marbrand), Allegiances (Lannister), Gender (1.0), Nobility (1.0), GoT (1.0), CoK (1.0), SoS (1.0), FfC (1.0), DwD (0.0), Dead (0.0)"
)

In [ ]:
code_context = """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create sample data
dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
np.random.seed(42)
df = pd.DataFrame({
    'date': dates,
    'sales': np.random.randint(100, 1000, size=len(dates)),
    'customer_id': np.random.randint(1, 101, size=len(dates)),
    'product_category': np.random.choice(['A', 'B', 'C', 'D'], size=len(dates))
})
"""

code = """
# Define a custom function
def calculate_category_metrics(group):
    return pd.Series({
        'total_sales': group['sales'].sum(),
        'avg_sales': group['sales'].mean(),
        'max_sales': group['sales'].max(),
        'unique_customers': group['customer_id'].nunique(),
        'sales_per_customer': group['sales'].sum() / group['customer_id'].nunique()
    })

# Apply the custom function
monthly_metrics = df.set_index('date').groupby([pd.Grouper(freq='M'), 'product_category']).apply(calculate_category_metrics).reset_index()

# Further data manipulation
monthly_metrics['month_year'] = monthly_metrics['date'].dt.strftime('%B %Y')
result = monthly_metrics.sort_values(['date', 'product_category']).reset_index(drop=True)

"""

In [ ]:
from src.analysis import analyse_code

analyse_code(code_context, code)